# **Laboratory 3 TDSE - Exploring Convolutional Layers Through Data and Experiments**
Juan Sebastian Puentes Julio

1. ## **PREPARATION**

In [2]:
%pip install numpy pandas matplotlib

In [3]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

##CHOSEN DATASET

## EXPLORING DATASET ANALYSIS